<a href="https://colab.research.google.com/github/123prashanth123/Fault-Detection-System/blob/Colabs/Build%20ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn, optim
from torchvision import models


# ResNet18 and ResNet34

In [ ]:
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, identity_downsample=None, stride=1):
        super(Block, self).__init__()

        self.identity_downsample = identity_downsample

        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=out_channels, eps=1e-5)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(num_features=out_channels, eps=1e-5)
    
    def forward(self, x):
        identity = x
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)
        x += identity
        x = self.relu(x)
        return x

In [ ]:
class BaseResNet(nn.Module):
    def __init__(self, block=None, num_residual_layers=None, image_channels=3, num_classes=1000):
        super(BaseResNet, self).__init__()

        self.in_channels = 64
        self.conv = nn.Conv2d(in_channels=image_channels, out_channels=self.in_channels, kernel_size=7, stride=2, padding=3)
        self.batchnorm = nn.BatchNorm2d(num_features=self.in_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.relu = nn.ReLU()

        self.layer1 = self.make_layers(block=block, num_residual_layers=num_residual_layers[0], out_channels=64, stride=1)
        self.layer2 = self.make_layers(block=block, num_residual_layers=num_residual_layers[1], out_channels=128, stride=2)
        self.layer3 = self.make_layers(block=block, num_residual_layers=num_residual_layers[2], out_channels=256, stride=2)
        self.layer4 = self.make_layers(block=block, num_residual_layers=num_residual_layers[3], out_channels=512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.fc = nn.Linear(in_features=512, out_features=num_classes)
    
    def make_layers(self, block=None, num_residual_layers=None, out_channels=None, stride=None):
        identity_downsample = None
        layers = []

        if stride != 1:
            identity_downsample = nn.Sequential(nn.Conv2d(in_channels=self.in_channels, out_channels=out_channels, kernel_size=1, stride=stride),
                                                nn.BatchNorm2d(num_features=out_channels, eps=1e-5))
        layers.append(block(in_channels=self.in_channels, out_channels=out_channels, identity_downsample=identity_downsample, stride=stride))
        self.in_channels = out_channels
        for i in range(num_residual_layers-1):
            layers.append(block(in_channels=self.in_channels, out_channels=out_channels))
        
        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.maxpool(self.batchnorm(self.conv(x))))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

In [ ]:
def ResNet18(image_channels=3, num_classes=1000):
    return ResNet(block=Block, num_layers_in_block=[2, 2, 2, 2], image_channels=image_channels, num_classes=num_classes)

def ResNet34(image_channels=3, num_classes=1000):
    return ResNet(block=Block, num_layers_in_block=[3, 4, 6, 3], image_channels=image_channels, num_classes=num_classes)

model = ResNet18()

# ResNet50, ResNet101, ResNet152

In [ ]:
class Block(nn.Module):
    def __init__(self, in_channels=None, out_channels=None, identity_downsample=None, stride=1):
        super(Block, self).__init__()
        
        self.expansion_factor = 4
        self.identity_downsample = identity_downsample
        self.relu = nn.ReLU()

        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1, stride=1, padding=0)
        self.batchnorm1 = nn.BatchNorm2d(num_features=out_channels, eps=1e-5)
        self.conv2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=stride, padding=1)
        self.batchnorm2 = nn.BatchNorm2d(num_features=out_channels, eps=1e-5)
        self.conv3 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels*self.expansion_factor, kernel_size=1, stride=1, padding=0)
        self.batchnorm3 = nn.BatchNorm2d(num_features=out_channels*self.expansion_factor, eps=1e-5)
    
    def forward(self, x):
        identity = x

        x = self.relu(self.batchnorm1(self.conv1(x)))
        x = self.relu(self.batchnorm2(self.conv2(x)))
        x = self.batchnorm3(self.conv3(x))

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)
        
        x += identity
        x = self.relu(x)
        return x

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block=None, num_layers_in_block=None, image_channels=None, num_classes=1000):
        super(ResNet, self).__init__()

        self.in_channels = 64
        self.relu = nn.ReLU()

        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))

        self.conv1 = nn.Conv2d(in_channels=image_channels, out_channels=self.in_channels, kernel_size=7, stride=2, padding=3)
        self.batchnorm1 = nn.BatchNorm2d(num_features=self.in_channels, eps=1e-5)

        self.layer1 = self.make_layers(block=block, num_layers_in_block=num_layers_in_block[0], out_channels=64, stride=1)
        self.layer2 = self.make_layers(block=block, num_layers_in_block=num_layers_in_block[1], out_channels=128, stride=2)
        self.layer3 = self.make_layers(block=block, num_layers_in_block=num_layers_in_block[2], out_channels=256, stride=2)
        self.layer4 = self.make_layers(block=block, num_layers_in_block=num_layers_in_block[3], out_channels=512, stride=2)

        self.fc = nn.Linear(in_features=512*4, out_features=num_classes)

    def make_layers(self, block=None, num_layers_in_block=None, out_channels=None, stride=None):
        identity_downsample = None
        layers = []

        if stride != 1 and self.in_channels != (out_channels*4):
            identity_downsample = nn.Sequential(nn.Conv2d(in_channels=self.in_channels, out_channels=out_channels*4, kernel_size=1, stride=stride),
                                                nn.BatchNorm2d(num_features=out_channels*4, eps=1e-5))
        
        layers.append(block(in_channels=self.in_channels, out_channels=out_channels, identity_downsample=identity_downsample, stride=stride))
        self.in_channels = out_channels*4

        for i in range(num_layers_in_block-1):
            layers.append(block(in_channels=self.in_channels, out_channels=out_channels))
        
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.maxpool(self.relu(self.batchnorm1(self.conv1(x))))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

In [ ]:
def ResNet50(image_channels=3, num_classes=1000):
    return ResNet(block=Block, num_layers_in_block=[3, 4, 6, 3], image_channels=image_channels, num_classes=num_classes)

def ResNet101(image_channels=3, num_classes=1000):
    return ResNet(block=Block, num_layers_in_block=[3, 4, 23, 3], image_channels=image_channels, num_classes=num_classes)

def ResNet152(image_channels=3, num_classes=1000):
    return ResNet(block=Block, num_layers_in_block=[3, 8, 36, 3], image_channels=image_channels, num_classes=num_classes)

model = ResNet50()